## GPT 2 End-to-end benchmarking

## GPT 2 Benchmark for HF

In [1]:
import torch
import time
from transformers import GPT2Tokenizer, GPT2LMHeadModel

/home/cfruan/.conda/envs/mlc-source-311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda:0"
# device = "mps"  # for M2 Ultra
tokenizer = GPT2Tokenizer.from_pretrained('/ssd1/cfruan/models/gpt2-medium')
model = GPT2LMHeadModel.from_pretrained('/ssd1/cfruan/models/gpt2-medium').to(device)

In [3]:
FULL_GETTYSBURG_ADDRESS = """
Four score and seven years ago our fathers brought forth on this continent, a new nation, conceived in Liberty, and dedicated to the proposition that all men are created equal.

Now we are engaged in a great civil war, testing whether that nation, or any nation so conceived and so dedicated, can long endure. We are met on a great battle-field of that war. We have come to dedicate a portion of that field, as a final resting place for those who here gave their lives that that nation might live. It is altogether fitting and proper that we should do this.

But, in a larger sense, we can not dedicate -- we can not consecrate -- we can not hallow -- this ground. The brave men, living and dead, who struggled here, have consecrated it, far above our poor power to add or detract. The world will little note, nor long remember what we say here, but it can never forget what they did here. It is for us the living, rather, to be dedicated here to the unfinished work which they who fought here have thus far so nobly advanced. It is rather for us to be here dedicated to the great task remaining before us -- that from these honored dead we take increased devotion to that cause for which they gave the last full measure of devotion -- that we here highly resolve that these dead shall not have died in vain -- that this nation, under God, shall have a new birth of freedom -- and that government of the people, by the people, for the people, shall not perish from the earth.
"""

In [5]:
input = tokenizer(FULL_GETTYSBURG_ADDRESS)
input_ids = torch.LongTensor(input["input_ids"]).unsqueeze(0).to(model.device)

# Prefill
logits = model.forward(input_ids)["logits"][0,-1].detach()
next_token = torch.multinomial(torch.nn.Softmax()(logits), 1)

# Decode
GEN_LEN = 512
decode_start = time.time()
for _ in range(GEN_LEN):
    logits = model.forward(next_token)["logits"][-1].detach()
    next_token = torch.multinomial(torch.nn.Softmax()(logits), 1)
decode_end = time.time()

/home/cfruan/.conda/envs/mlc-source-311/lib/python3.11/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [6]:
print("Decode speed: %.3f tok/s" % (GEN_LEN / (decode_end - decode_start)))

Decode speed: 75.165 tok/s


### CUDA benchmark for each TVM optimization

We run the following tests by running the command: 

`python -m benchmark_gpt2 --model-lib ./dist/gpt2-xl-q0f32/gpt2-xl-q0f32-cuda.so --device cuda --generate-length 512`

We substitute the model library with the one we want to benchmark. Each model library has a different dlight schedule. For each model library, we prefill the entire gettysburg address (322 tokens), and let the model generate 64 tokens.

The results are the following:
- Full schedule: `gpt2-xl-q0f32-full-cuda.so`
  - Prefill: 4419 tok/s
  - Decode: 102.9 tok/s
- Base schedule: `gpt2-xl-q0f32-base-cuda.so`
  - Prefill: 70.0 tok/s
  - Decode: 3.5 tok/s
- Matmul-only schedule: `gpt2-xl-q0f32-matmul-cuda.so`
  - Prefill: 3321.4 tok/s
  - Decode: 3.5 tok/s
- GEMV-only schedule: `gpt2-xl-q0f32-gemv-cuda.so`
  - Prefill: 70.1 tok/s
  - Decode: 53.5 tok/s
- Reduction-only schedule: `gpt2-xl-q0f32-reduction-cuda.so`
  - Prefill: 70.1 tok/s
  - Decode: 60.7 tok/s

Across platforms:
- MacOS w/ Intel Chip:
  - Prefill: 28.9 tok/s
  - Decode: 1.1 tok/s
- WebGPU on the same MacOS: 
  - Prefill 10.96 tok/s
  - Decode: 0.81 tok/s

There are several observations: optimizing matrix multiplication alone is enough to 

It makes sense that optimizing matrix multiplication only is enough to bump a significant amount of 